In [1]:
import numpy as np
import pandas as pd
import operator

The function below defines the ratio of NaNs in the code to the length of the column. If this ratio is greater than given threshold, the column is considered as severely missing values.

In [2]:
# count number of seriously missing values
def missing_values(data, threshold):
    i=0
    names = []
    data_length = len(data)
    for name in data.columns:
        a = ((1-data[name].count()/data_length)*100).astype(int)
        if a > threshold:
            i+=1
            names.append(name)
    return(i, names)

In [3]:
data = pd.read_csv('https://www.openml.org/data/get_csv/53995/KDDCup09_churn.arff')
data = data.replace({"?": np.NaN})

In [4]:
bad_columns = missing_values(data,90)[1]

In [5]:
data_clean = data.copy()    

In [6]:
for col in bad_columns:
    data_clean[col+"_na"] = data_clean[col] == np.NaN
    data_clean.drop(col,axis=1,inplace=True)

In [7]:
data_clean.head(5)

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var186_na,Var187_na,Var188_na,Var190_na,Var191_na,Var209_na,Var213_na,Var215_na,Var224_na,Var230_na
0,1526,7,184,464,580,14,128,166.56,0,3570,...,False,False,False,False,False,False,False,False,False,False
1,525,0,0,168,210,2,24,353.52,0,4764966,...,False,False,False,False,False,False,False,False,False,False
2,5236,7,904,1212,1515,26,816,220.08,0,5883894,...,False,False,False,False,False,False,False,False,False,False
3,NaN,0,0,NaN,0,NaN,0,22.08,0,0,...,False,False,False,False,False,False,False,False,False,False
4,1029,7,3216,64,80,4,64,200,0,0,...,False,False,False,False,False,False,False,False,False,False


In [8]:
for col, i in zip(data_clean.columns,data_clean.dtypes):
    if i == object:
        try:
            data_clean[col] = data_clean[col].astype(float)
        except:
            pass
str_cols = []
for col in data_clean.columns:
    for i in range(len(data_clean)):
        if pd.isna(data_clean[col].values[i]) == False:
            if(type(data_clean[col].values[i])==str):
                str_cols.append(col)
            break

In [32]:
def max_occurency(a):
    '''this function returns the most frequent element 
    in the given iterable element, while NaN values are ignored.
    '''
    freq_dict = {}
    for elem in a:
        if pd.isna(elem) == False:
            if freq_dict.get(elem) == None:
                freq_dict[elem] = 1
            else:
                freq_dict[elem] += 1
    return max(freq_dict, key=freq_dict.get)

In [19]:
def freq_occurency(a):
    freq_dict = {}
    for elem in a:
        if freq_dict.get(elem) == None:
            freq_dict[elem] = 1
        else:
            freq_dict[elem] += 1
    return freq_dict

We apply here function max_occurency to string columns in order to fill NaNs with most frequent value in the corresponding column.

In [20]:
for col in str_cols:
    string_freq = max_occurency(data_clean[col].values)
    data_clean[col].fillna(string_freq, inplace=True)

In [23]:
freq_dicts = []
for col in str_cols:
    freq_dicts.append(freq_occurency(data_clean[col].values))

In [24]:
frequent_strs = []
for dictionary in freq_dicts:
    strs = sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True)[:3]
    #print(strs)
    frequent_strs.append(strs)

This function is used for decreasing the number of distinct string values. Here only most frequent values are kept, while other values are set to -1. After that we can either proceed one-hot encoding to them or just simply leave them as they are 
$(vals \in{-1,0,1,2})$.

In [25]:
def prepare_repl_dict(freq_dict,i):
    try:
        repl_dict = [freq_dict[i][0][0],
                 freq_dict[i][1][0],
                 freq_dict[i][2][0]]
        repl_set = {freq_dict[i][0][0],
                freq_dict[i][1][0],
                freq_dict[i][2][0]}
    except:
        repl_dict = [freq_dict[i][0][0],
                 freq_dict[i][1][0]]
        repl_set = {freq_dict[i][0][0],
                freq_dict[i][1][0]}
    return repl_dict, repl_set

Here we apply our function to the data keeping in mind that exceptions may occur (if number of unique values is less than 3)

In [26]:
for i in range(len(str_cols)):
    r_array, r_set = prepare_repl_dict(frequent_strs,i)
    data_clean[str_cols[i]][data_clean[str_cols[i]].isin(r_set) == False] = -1
    try:
        for j in range(3):
            data_clean[str_cols[i]][data_clean[str_cols[i]] == r_array[j]] = j
    except:
        for j in range(2):
            data_clean[str_cols[i]][data_clean[str_cols[i]] == r_array[j]] = j

/Users/egor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/egor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/egor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
data_clean = data_clean.astype(float)
for col in data_clean.columns:
    med = np.ceil(np.median(data_clean[col].values))
    data_clean[col].fillna(med)

/Users/egor/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [28]:
for col in data_clean.columns:
    data_clean[col]=data_clean[col].fillna(np.ceil(np.median(data_clean[col][data_clean[col].isna()==False].values)))

Let's ensure that we don't have any NaNs left:

In [29]:
data_clean.count()

Var6         50000
Var7         50000
Var13        50000
Var21        50000
Var22        50000
Var24        50000
Var25        50000
Var28        50000
Var35        50000
Var38        50000
Var44        50000
Var57        50000
Var65        50000
Var72        50000
Var73        50000
Var74        50000
Var76        50000
Var78        50000
Var81        50000
Var83        50000
Var85        50000
Var94        50000
Var109       50000
Var112       50000
Var113       50000
Var119       50000
Var123       50000
Var125       50000
Var126       50000
Var132       50000
             ...  
Var164_na    50000
Var165_na    50000
Var166_na    50000
Var167_na    50000
Var168_na    50000
Var169_na    50000
Var170_na    50000
Var171_na    50000
Var172_na    50000
Var174_na    50000
Var175_na    50000
Var176_na    50000
Var177_na    50000
Var178_na    50000
Var179_na    50000
Var180_na    50000
Var182_na    50000
Var183_na    50000
Var184_na    50000
Var185_na    50000
Var186_na    50000
Var187_na   

In [30]:
data_clean[str_cols] = data_clean[str_cols].astype(int)